<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">

<p>از آنجا که .NET به صورت داخلی از POP3 پشتیبانی نمی‌کند، باید خودتان برنامه‌نویسی سطح TCP را انجام دهید تا بتوانید از سرور POP3 ایمیل دریافت کنید. در ادامه، یک مکالمه POP3 به طور مختصر شرح داده شده است.</p>

<h5>مکالمه POP3</h5>
<p>در اینجا چگونگی ارتباط یک کلاینت با سرور POP3 به صورت مرحله به مرحله شرح داده شده است:</p>

<ol><li><p><strong>اتصال کلاینت به سرور</strong>:</p><ul><li><strong>کلاینت</strong>: به سرور متصل می‌شود.</li><li><strong>سرور</strong>: <code>+OK Hello there.</code></li><li><strong>توضیح</strong>: پیام خوش‌آمدگویی از طرف سرور.</li></ul></li><li><p><strong>احراز هویت کاربر</strong>:</p><ul><li><p><strong>کلاینت</strong>: <code>USER joe</code></p></li><li><p><strong>سرور</strong>: <code>+OK Password required.</code></p></li><li><p><strong>توضیح</strong>: درخواست نام کاربری و رمز عبور.</p></li><li><p><strong>کلاینت</strong>: <code>PASS password</code></p></li><li><p><strong>سرور</strong>: <code>+OK Logged in.</code></p></li><li><p><strong>توضیح</strong>: تایید ورود موفقیت‌آمیز.</p></li></ul></li><li><p><strong>لیست کردن پیام‌ها</strong>:</p><ul><li><strong>کلاینت</strong>: <code>LIST</code></li><li><strong>سرور</strong>:
<code>
+OK  
1 1876  
2 5412  
3 845  
.

</code>
</li><li><strong>توضیح</strong>: لیست پیام‌ها با شماره ID و اندازه فایل هر پیام.</li></ul></li><li><p><strong>دریافت یک پیام خاص</strong>:</p><ul><li><strong>کلاینت</strong>: <code>RETR 1</code></li><li><strong>سرور</strong>:<code>+OK 1876 octets
Content of message #1...
.
</code> </li><li><strong>توضیح</strong>: دریافت محتوای پیام با شماره ID مشخص شده.</li></ul></li><li><p><strong>حذف یک پیام</strong>:</p><ul><li><strong>کلاینت</strong>: <code>DELE 1</code></li><li><strong>سرور</strong>: <code>+OK Deleted.</code></li><li><strong>توضیح</strong>: حذف پیام با شماره ID مشخص شده.</li></ul></li><li><p><strong>خروج از سرور</strong>:</p><ul><li><strong>کلاینت</strong>: <code>QUIT</code></li><li><strong>سرور</strong>: <code>+OK Bye-bye.</code></li><li><strong>توضیح</strong>: پایان مکالمه و قطع ارتباط.</li></ul></li></ol>
</div>

In [ ]:
using System;
using System.Net.Sockets;
using System.Text;
using System.Threading.Tasks;

class Program
{
    static async Task Main(string[] args)
    {
        string server = "pop3.example.com";
        int port = 110;
        string user = "joe";
        string password = "password";

        using TcpClient client = new TcpClient();
        await client.ConnectAsync(server, port);

        using NetworkStream stream = client.GetStream();
        byte[] buffer = new byte[1024];
        int bytesRead;

        bytesRead = await stream.ReadAsync(buffer, 0, buffer.Length);
        Console.WriteLine(Encoding.ASCII.GetString(buffer, 0, bytesRead));

        await SendCommandAsync(stream, $"USER {user}\r\n");
        await ReadResponseAsync(stream);

        await SendCommandAsync(stream, $"PASS {password}\r\n");
        await ReadResponseAsync(stream);

        await SendCommandAsync(stream, "LIST\r\n");
        await ReadResponseAsync(stream);

        await SendCommandAsync(stream, "RETR 1\r\n");
        await ReadResponseAsync(stream);

        await SendCommandAsync(stream, "DELE 1\r\n");
        await ReadResponseAsync(stream);

        await SendCommandAsync(stream, "QUIT\r\n");
        await ReadResponseAsync(stream);
    }

    static async Task SendCommandAsync(NetworkStream stream, string command)
    {
        byte[] buffer = Encoding.ASCII.GetBytes(command);
        await stream.WriteAsync(buffer, 0, buffer.Length);
    }

    static async Task ReadResponseAsync(NetworkStream stream)
    {
        byte[] buffer = new byte[1024];
        int bytesRead = await stream.ReadAsync(buffer, 0, buffer.Length);
        Console.WriteLine(Encoding.ASCII.GetString(buffer, 0, bytesRead));
    }
}
